In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)


# Differential Photometry Preview  

In [ ]:
import vso.util
import vso.data

OBJ_NAME='SZ Lyn'
SESSION_TAG='20250126'
# OBJ_NAME='RZ Lyr'
# SESSION_TAG='20240827'
IMAGE_ROOT = '/srv/public/img'
WORK_ROOT = '/srv/public'

session = vso.util.Session(tag=SESSION_TAG, name=OBJ_NAME)
layout = vso.util.WorkLayout(WORK_ROOT)
session_layout = layout.get_session(session)
settings = vso.util.Settings(session_layout.settings_file_path)

In [ ]:
from astropy.table import join
# tr = QTable.read(session_layout.root_dir / "Tr_B_V.ecsv")
from vsopy import phot
bands = ('B', 'V')

provider = phot.BatchDataProvider(session_layout)
tr = join(phot.batch_create_simple_transform(provider, bands), provider.batches_, 'batch_id')

In [ ]:
tr['xfm']['Tab']['val']

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats as sst

var = 'Tab'
var_err = var + '_err'

trvar = tr['xfm']['Tab']['val']
trerr = tr['xfm']['Tab']['err']

lr = sst.linregress(tr['airmass'], trvar)
line = np.array([np.min(tr['airmass']), np.max(tr['airmass'])])
res = trvar - (tr['airmass'] * lr.slope + lr.intercept)
res_std = np.std(res)
dev_lim = 2

fltr = np.abs(res) < dev_lim*res_std
#fltr = np.abs(tr[var_err]) < np.percentile(tr[var_err], 75)

lr1 = sst.linregress(tr['airmass'][fltr], trvar[fltr])
res1 = trvar[fltr] - (tr['airmass'][fltr] * lr1.slope + lr1.intercept)
res_std_1 = np.std(res1)
#lr.stderr, lr1.stderr


In [ ]:

def regr(lim):
    fltr = np.abs(res) < lim*res_std
    lrn = sst.linregress(tr['airmass'][fltr], trvar[fltr])
    return lrn.slope, lrn.stderr, lrn.intercept, lrn.intercept_stderr

np.vectorize(regr)(np.linspace(.25, 2.0, 8))

In [ ]:
sst.normaltest(res), sst.normaltest(res1)

In [ ]:

def reg_label(var, lr):
    return f'${{{var}}} = x ({{{lr.slope:.2g}}} \\pm {{{lr.stderr:.2g}}}) + ({{{lr.intercept:.2g}}} \\pm {{{lr.intercept_stderr:.2g}}}), R^2 = {{{lr.rvalue ** 2:.2g}}}$'

fig = plt.figure(figsize=(12.80, 8.00))
gs = fig.add_gridspec(2, 2, width_ratios=[2, 1])
ax = fig.add_subplot(gs[0, 0])

#ax.plot(tr['airmass'][fltr], tr[var][fltr], '.')
ax.errorbar(tr['airmass'][fltr], trvar[fltr], yerr=trerr[fltr], fmt='.')
ax.plot(line, line * lr.slope + lr.intercept, "-")
ax.plot(line, line * lr.slope + lr.intercept + dev_lim*res_std, "--", color='grey')
ax.plot(line, line * lr.slope + lr.intercept - dev_lim*res_std, "--", color='grey')
#ax.plot(tr['airmass'][~fltr], tr[var][~fltr], '.', color='red')
ax.errorbar(tr['airmass'][~fltr], trvar[~fltr], yerr=trerr[~fltr], fmt='.', color='red')
ax.set_ylabel(var)
ax.set_xlabel('Air mass')
ax.set_title(reg_label(var, lr))

ax1 = fig.add_subplot(gs[0, 1])
# ax1.hist(res, bins=20)
sst.probplot(res, dist="norm", plot=ax1)

ax2 = fig.add_subplot(gs[1, 0])
ax2.plot(tr['airmass'][fltr], trvar[fltr], '.')
ax2.plot(line, line * lr1.slope + lr1.intercept, "-")
ax2.plot(line, line * lr1.slope + lr1.intercept + dev_lim*res_std_1, "--", color='grey')
ax2.plot(line, line * lr1.slope + lr1.intercept - dev_lim*res_std_1, "--", color='grey')
ax2.set_ylabel(var)
ax2.set_xlabel('Air mass')
ax2.set_title(reg_label(var, lr1))

ax3 = fig.add_subplot(gs[1, 1])
#ax3.hist(res1, bins=20)
sst.probplot(res1, dist="norm", plot=ax3)

plt.tight_layout()
plt.show()


In [ ]:
# %matplotlib ipympl
# #fig = plt.figure(figsize=(8.00, 16.00))
# gs = fig.add_gridspec(1, 1)
# ax = fig.add_subplot(gs[0, 0], projection='3d')
# ax.scatter(tr['Ta'], tr['Tb'], tr['Tab'], marker='.')
# ax.set_xlabel('Ta')
# ax.set_ylabel('Tb')
# ax.set_zlabel('Tab')
# plt.show()


In [ ]:
fig = plt.figure(figsize=(12.80, 8.00))
gs = fig.add_gridspec(1, 3)
ax = fig.add_subplot(gs[0, 0])
ax.scatter(tr['xfm']['Ta']['val'], tr['xfm']['Tb']['val'], marker='.')
ax1 = fig.add_subplot(gs[0, 1])
ax1.scatter(tr['xfm']['Ta']['val'], tr['xfm']['Tab']['val'], marker='.')
ax2 = fig.add_subplot(gs[0, 2])
ax2.scatter(tr['xfm']['Tb']['val'], tr['xfm']['Tab']['val'], marker='.')
plt.show()


In [ ]:
fig = plt.figure(figsize=(8.00, 16.00))
gs = fig.add_gridspec(3, 1)

ax = fig.add_subplot(gs[0, 0])
ax.plot(tr['airmass'], tr[var_err], '.')
ax.set_ylabel(var_err)
ax.set_xlabel('Air mass')

ax1 = fig.add_subplot(gs[1, 0])
ax1.plot(tr['id'], tr[var_err], '.')
ax1.set_ylabel(var_err)
#ax1.set_xlabel('Air mass')

ax2 = fig.add_subplot(gs[2, 0])
ax2.plot(tr['id'], tr['airmass'], '.')
ax2.set_ylabel('Air mass')

plt.show()